<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/Finetune_vs_Finetune_after_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

In [ ]:
! pip install transformers
! pip install datasets

In [ ]:
! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git/
%cd PyTorch-Architectures/

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from toolkit.custom_dataset_nlp import DataLoaderTextClassification
from toolkit.utils import get_linear_schedule_with_warmup
from toolkit.metrics import nlp_compute_accuracy
from tqdm.auto import tqdm

In [3]:
# Hyperparameters
BS = 128
EPOCHS = 3
LR = 5e-4

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)

In [5]:
dataset_train = DataLoaderTextClassification(tokenizer, train=True)
dataset_valid = DataLoaderTextClassification(tokenizer, train=False)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [6]:
train_loader = dataset_train.return_dataloader(batch_size=BS, shuffle=True)
valid_loader = dataset_valid.return_dataloader(batch_size=BS, shuffle=False)
print('Length of Train Loader: ', len(train_loader))
print('Length of Valid Loader: ', len(valid_loader))

Length of Train Loader:  527
Length of Valid Loader:  7


In [7]:
# Sanity check forward pass
model.eval()
for sample in train_loader:
  outputs = model(input_ids=sample['input_ids'].to(device),
                  attention_mask=sample['attention_mask'].to(device),
                  labels=sample['labels'].to(device))
  print(outputs.loss.item())
  break

0.6934221386909485


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, 
                                            num_training_steps=len(train_loader) * EPOCHS)

In [9]:
progress_bar = tqdm(range(len(train_loader) * EPOCHS))

for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    outputs = model(input_ids=sample['input_ids'].to(device),
                    attention_mask=sample['attention_mask'].to(device),
                    labels=sample['labels'].to(device))

    loss = outputs.loss
    loss.backward()

    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)
  model.eval()
  with torch.set_grad_enabled(False):
    train_acc = nlp_compute_accuracy(model, train_loader, device)
    valid_acc = nlp_compute_accuracy(model, valid_loader, device)
  print('Train Accuracy: %.2f%% || Valid Accuracy: %.2f%%' % (train_acc, valid_acc))

Train Accuracy: 91.29% || Valid Accuracy: 76.15%
Train Accuracy: 95.34% || Valid Accuracy: 77.18%
Train Accuracy: 96.86% || Valid Accuracy: 77.18%
